# Spike Pattern Identifier - Compact Version

Interactive notebook for analyzing neural spike patterns with configurable parameters.

## 1. Setup and Data Loading

In [ ]:
# Import libraries and load data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

from utils import SpikePatternAnalyzer, load_spike_data, load_electrode_config, print_data_summary

# Load data
spike_data = load_spike_data('spikes.csv')
electrode_config = load_electrode_config('electrode.cfg')
analyzer = SpikePatternAnalyzer(spike_data, electrode_config)

# Print summary
print_data_summary(spike_data, electrode_config)

# Get unique units for selection
unique_units = sorted(spike_data['unit'].unique())
print(f"\nAvailable units: {unique_units[:10]}..." if len(unique_units) > 10 else f"\nAvailable units: {unique_units}")

## 2. Analysis Parameters

In [ ]:
# Create parameter widgets
unit_selector = widgets.Dropdown(
    options=unique_units,
    value=unique_units[0],
    description='Unit:'
)

burst_max_isi = widgets.FloatSlider(
    value=0.01, min=0.001, max=0.1, step=0.001,
    description='Burst Max ISI (s):'
)

burst_min_spikes = widgets.IntSlider(
    value=3, min=2, max=10,
    description='Min Spikes in Burst:'
)

fr_window = widgets.FloatSlider(
    value=1.0, min=0.1, max=5.0, step=0.1,
    description='FR Window (s):'
)

# Analysis controls
analyze_button = widgets.Button(
    description='Run Analysis',
    button_style='success'
)

visualize_button = widgets.Button(
    description='Create Plots',
    button_style='info'
)

# Display controls
controls = widgets.VBox([
    widgets.HTML("<h3>Analysis Parameters</h3>"),
    unit_selector,
    burst_max_isi,
    burst_min_spikes,
    fr_window,
    widgets.HBox([analyze_button, visualize_button])
])

display(controls)

## 3. Analysis and Visualization

In [ ]:
# Global variables for results
current_results = None
current_unit = None

def run_analysis(button):
    global current_results, current_unit
    
    with output_area:
        clear_output(wait=True)
        
        current_unit = unit_selector.value
        params = {
            'burst_max_isi': burst_max_isi.value,
            'burst_min_spikes': burst_min_spikes.value,
            'firing_rate_window': fr_window.value,
        }
        
        print(f"Analyzing Unit {current_unit}...")
        current_results = analyzer.run_pattern_analysis(current_unit, params)
        analyzer.print_analysis_results(current_results, current_unit)

def create_plots(button):
    global current_results, current_unit
    
    if current_results is None:
        print("Run analysis first!")
        return
        
    print(f"Creating plots for Unit {current_unit}...")
    
    try:
        from utils import plot_unit_analysis
        fig = plot_unit_analysis(analyzer, current_unit, current_results)
        plt.show()
        print("Plots created successfully!")
    except Exception as e:
        print(f"Error creating plots: {e}")

# Connect buttons
analyze_button.on_click(run_analysis)
visualize_button.on_click(create_plots)

# Output area
output_area = widgets.Output()
display(output_area)

## 4. Batch Analysis

In [ ]:
# Batch analysis
batch_units = widgets.SelectMultiple(
    options=unique_units[:20],  # Limit to first 20 units
    value=unique_units[:5],
    description='Units:',
    rows=8
)

batch_button = widgets.Button(
    description='Run Batch Analysis',
    button_style='primary'
)

def run_batch(button):
    selected_units = list(batch_units.value)
    if not selected_units:
        print("Select units first!")
        return
        
    print(f"Running batch analysis on {len(selected_units)} units...")
    
    params = {
        'burst_max_isi': burst_max_isi.value,
        'burst_min_spikes': burst_min_spikes.value,
        'firing_rate_window': fr_window.value,
    }
    
    batch_results = {}
    for unit_id in selected_units:
        try:
            results = analyzer.run_pattern_analysis(unit_id, params)
            batch_results[unit_id] = results
        except Exception as e:
            batch_results[unit_id] = {'error': str(e)}
    
    # Print summary using utils function
    analyzer.print_batch_summary(batch_results)

batch_button.on_click(run_batch)

batch_controls = widgets.VBox([
    widgets.HTML("<h3>Batch Analysis</h3>"),
    batch_units,
    batch_button
])

display(batch_controls)

## 5. Export Results

In [ ]:
# Export functionality
import json

export_button = widgets.Button(
    description='Export Current Results',
    button_style='warning'
)

def export_results(button):
    global current_results, current_unit
    
    if current_results is None:
        print("No results to export!")
        return
        
    filename = f"pattern_analysis_unit_{current_unit}.json"
    
    # Prepare export data (convert numpy arrays)
    export_data = {}
    for key, value in current_results.items():
        if hasattr(value, 'tolist'):
            export_data[key] = value.tolist()
        elif isinstance(value, dict):
            export_data[key] = {}
            for k, v in value.items():
                export_data[key][k] = v.tolist() if hasattr(v, 'tolist') else v
        else:
            export_data[key] = value
    
    with open(filename, 'w') as f:
        json.dump(export_data, f, indent=2, default=str)
    
    print(f"Results exported to {filename}")

export_button.on_click(export_results)
display(export_button)